In [1]:
import requests
import pandas as pd

In [2]:

# Utilisation de pd.set_option pour afficher toutes les colonnes du DataFrame
pd.set_option('display.max_columns', None)

station_names = ["Paris Bercy", "Paris Saint-Lazare", "Paris Gare de Lyon", "Paris Gare du Nord", "Paris Montparnasse", "Paris Est", "Paris Austerlitz"]

records = []

for name in station_names:
    for year in range(2019, 2023):
        url = f"https://ressources.data.sncf.com/api/records/1.0/search/?dataset=objets-trouves-restitution&q=Paris&rows=10000&sort=date&facet=date&facet=gc_obo_gare_origine_r_name&facet=gc_obo_type_c&refine.gc_obo_gare_origine_r_name={name.replace(' ', '+')}&refine.date={year}"
        response = requests.get(url)
        data = response.json()
        records += data['records']

df1 = pd.json_normalize(records)
df1 = df1.loc[:, ['fields.date', 'fields.gc_obo_gare_origine_r_name', 'fields.gc_obo_type_c']]
df1['fields.date'] = pd.to_datetime(df1['fields.date']).dt.strftime('%Y-%m-%d')
# création du CSV 
df1.to_csv("objets-trouves.csv", index=False)


In [3]:
df1

,fields.date,fields.gc_obo_gare_origine_r_name,fields.gc_obo_type_c
0,2019-12-29,Paris Bercy,"Porte-monnaie / portefeuille, argent, titres"
1,2019-12-26,Paris Bercy,"Bagagerie: sacs, valises, cartables"
2,2019-12-26,Paris Bercy,"Bagagerie: sacs, valises, cartables"
3,2019-12-26,Paris Bercy,"Bagagerie: sacs, valises, cartables"
4,2019-12-26,Paris Bercy,"Vêtements, chaussures"
...,...,...,...
81627,2022-01-04,Paris Austerlitz,"Porte-monnaie / portefeuille, argent, titres"
81628,2022-01-04,Paris Austerlitz,"Porte-monnaie / portefeuille, argent, titres"
81629,2022-01-04,Paris Austerlitz,"Bagagerie: sacs, valises, cartables"
81630,2022-01-04,Paris Austerlitz,"Bagagerie: sacs, valises, cartables"


In [4]:
import sqlite3
connexion= sqlite3.connect("bdd.db")
curseur = connexion.cursor()
curseur.execute('''CREATE TABLE IF NOT EXISTS df1 (
              date TEXT NOT NULL,
              gare TEXT NOT NULL,
              feels_like FLOAT,
              type_objet TEXT
              );''')

connexion.commit()

query = "INSERT INTO df1 (date, gare, type_objet) VALUES (?, ?, ?)"
curseur.executemany(query, df1.values.tolist())

# Validation de la transaction
connexion.commit()

# Fermeture de la connexion
connexion.close()

**DATA FRAME objet perdu 2019**


In [5]:
df1['fields.date'] = pd.to_datetime(df1['fields.date'])
df1_2019 = df1[df1['fields.date'].dt.year == 2019]
df1_2019

,fields.date,fields.gc_obo_gare_origine_r_name,fields.gc_obo_type_c
0,2019-12-29,Paris Bercy,"Porte-monnaie / portefeuille, argent, titres"
1,2019-12-26,Paris Bercy,"Bagagerie: sacs, valises, cartables"
2,2019-12-26,Paris Bercy,"Bagagerie: sacs, valises, cartables"
3,2019-12-26,Paris Bercy,"Bagagerie: sacs, valises, cartables"
4,2019-12-26,Paris Bercy,"Vêtements, chaussures"
...,...,...,...
80380,2019-01-03,Paris Austerlitz,"Appareils électroniques, informatiques, appare..."
80381,2019-01-03,Paris Austerlitz,"Clés, porte-clés, badge magnétique"
80382,2019-01-03,Paris Austerlitz,"Bagagerie: sacs, valises, cartables"
80383,2019-01-03,Paris Austerlitz,"Appareils électroniques, informatiques, appare..."


In [6]:
#df_provisoire_2019['fields.date'] = pd.to_datetime(df_provisoire_['fields.date'])
df1_2019['fields.date']=df1_2019['fields.date'].dt.strftime('%U')

/tmp/ipykernel_330594/2308099575.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_2019['fields.date']=df1_2019['fields.date'].dt.strftime('%U')


In [7]:
df1_2019

,fields.date,fields.gc_obo_gare_origine_r_name,fields.gc_obo_type_c
0,52,Paris Bercy,"Porte-monnaie / portefeuille, argent, titres"
1,51,Paris Bercy,"Bagagerie: sacs, valises, cartables"
2,51,Paris Bercy,"Bagagerie: sacs, valises, cartables"
3,51,Paris Bercy,"Bagagerie: sacs, valises, cartables"
4,51,Paris Bercy,"Vêtements, chaussures"
...,...,...,...
80380,00,Paris Austerlitz,"Appareils électroniques, informatiques, appare..."
80381,00,Paris Austerlitz,"Clés, porte-clés, badge magnétique"
80382,00,Paris Austerlitz,"Bagagerie: sacs, valises, cartables"
80383,00,Paris Austerlitz,"Appareils électroniques, informatiques, appare..."


In [8]:
df1_2019.to_csv("objets-trouves_2019.csv", index=False)


In [9]:
df1_2019['fields.date'].value_counts().sort_index()

00    364
01    456
02    450
03    511
04    449
05    532
06    576
07    483
08    560
09    537
10    436
11    567
12    551
13    519
14    472
15    567
16    535
17    482
18    526
19    526
20    524
21    496
22    584
23    554
24    563
25    579
26    627
27    597
28    677
29    541
30    450
31    540
32    562
33    552
34    612
35    593
36    481
37    645
38    479
39    422
40    576
41    459
42    547
43    504
44    594
45    448
46    509
47    504
48    371
49    188
50    283
51    259
52    142
Name: fields.date, dtype: int64

**DATA FRAME objet perdu  2020**

In [10]:
df1['fields.date'] = pd.to_datetime(df1['fields.date'])
df1_2020 = df1[df1['fields.date'].dt.year == 2020]
df1_2020['fields.date']=df1_2020['fields.date'].dt.strftime('%U')

/tmp/ipykernel_330594/2798880336.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_2020['fields.date']=df1_2020['fields.date'].dt.strftime('%U')


In [11]:
df1_2020

,fields.date,fields.gc_obo_gare_origine_r_name,fields.gc_obo_type_c
570,52,Paris Bercy,"Bagagerie: sacs, valises, cartables"
571,52,Paris Bercy,Pièces d'identités et papiers personnels
572,52,Paris Bercy,"Bagagerie: sacs, valises, cartables"
573,52,Paris Bercy,Optique
574,51,Paris Bercy,"Bagagerie: sacs, valises, cartables"
...,...,...,...
80619,01,Paris Austerlitz,"Vélos, trottinettes, accessoires 2 roues"
80620,01,Paris Austerlitz,"Vélos, trottinettes, accessoires 2 roues"
80621,00,Paris Austerlitz,Pièces d'identités et papiers personnels
80622,00,Paris Austerlitz,Pièces d'identités et papiers personnels


In [12]:
df1_2020.to_csv("objets-trouves_2020.csv", index=False)

**DATA FRAME objet perdu 2021**


In [13]:
df1['fields.date'] = pd.to_datetime(df1['fields.date'])
df1_2021 = df1[df1['fields.date'].dt.year == 2021]
df1_2021['fields.date']=df1_2021['fields.date'].dt.strftime('%U')

/tmp/ipykernel_330594/764819555.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_2021['fields.date']=df1_2021['fields.date'].dt.strftime('%U')


In [14]:
df1_2021.to_csv("objets-trouves_2021.csv", index=False)

**DATA FRAME objet perdu 2022**

In [15]:
df1['fields.date'] = pd.to_datetime(df1['fields.date'])
df1_2022 = df1[df1['fields.date'].dt.year == 2022]
df1_2022['fields.date']=df1_2022['fields.date'].dt.strftime('%U')

/tmp/ipykernel_330594/3772376561.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_2022['fields.date']=df1_2022['fields.date'].dt.strftime('%U')


In [16]:
df1_2022.to_csv("objets-trouves_2022.csv", index=False)

**DATA FRAME objet perdu par jour**

In [57]:
df1_jour = df1.groupby(pd.Grouper(key='fields.date', freq='D')).count()
df1_jour = df1_jour.reset_index()
df1_jour['objet_jour'] = df1_jour['fields.gc_obo_type_c']
df1_jour[['fields.date', 'objet_jour']].to_csv('objet-perdu-jour.csv')

**DATA FRAME TEMPERATURE**

In [18]:
df_temperature = pd.read_csv("temperature.csv")
# Prendre le DataFrame jusqu à la ligne 35064
df_temperature = df_temperature[:35064]

# # Sauvegarder le fichier CSV
# df_temperature.to_csv('temperature.csv', index=False)
# df_temperature = pd.read_csv("temperature.csv")

df_temperature = df_temperature[['time', 'temperature_2m (°C)']]
df_temperature


,time,temperature_2m (°C)
0,2019-01-01T00:00,7.4
1,2019-01-01T01:00,7.2
2,2019-01-01T02:00,7.1
3,2019-01-01T03:00,6.9
4,2019-01-01T04:00,6.7
...,...,...
35059,2022-12-31T19:00,14.3
35060,2022-12-31T20:00,14.9
35061,2022-12-31T21:00,15.2
35062,2022-12-31T22:00,15.1


In [61]:
# Convertir la colonne "time" en type de données datetime
df_temperature['time'] = pd.to_datetime(df_temperature['time'])

# Regrouper les données par jour : freq = 'D'
df_temperature_jour = df_temperature.groupby(pd.Grouper(key='time', freq='D')).mean(numeric_only=True)

# # # Afficher les moyennes des températures par jour
df_temperature_jour

,temperature_2m (°C)
time,
2019-01-01,7.558333
2019-01-02,5.420833
2019-01-03,2.325000
2019-01-04,1.500000
2019-01-05,2.875000
...,...
2022-12-27,5.654167
2022-12-28,10.150000
2022-12-29,9.591667


In [66]:
df_temperature_jour =  df_temperature_jour.reset_index()

In [68]:
df_temperature_jour

,time,temperature_2m (°C)
0,2019-01-01,7.558333
1,2019-01-02,5.420833
2,2019-01-03,2.325000
3,2019-01-04,1.500000
4,2019-01-05,2.875000
...,...,...
1456,2022-12-27,5.654167
1457,2022-12-28,10.150000
1458,2022-12-29,9.591667
1459,2022-12-30,9.904167


In [38]:
# mettre à jour / index = True
df_temperature_jour.to_csv('temperature_jour.csv')

**DATA FRAME FREQUENTATION**